In [1]:
import numpy as np
from random import randint
from numpy.core.numeric import Inf

In [2]:
def create_env(n_rows, n_cols):
    env = np.full((n_rows, n_cols), '-')
    env = add_margin(env)
    env = add_vehicles(env)
    env = add_player(env)
    return env

In [3]:
def add_vehicles(env):
    n_rows = env.shape[0]
    n_cols = env.shape[1]
    for row in range(1, n_rows-1):
        column_with_vehicle = randint(0, n_cols-1)
        env[row, column_with_vehicle] = 'O'
    return env    

In [4]:
def add_player(env):
    mid_col = env.shape[1] // 2
    env[0,mid_col] = 'A'
    return env

In [5]:
def add_margin(env):
    env[0] = 'I'
    env[-1] = 'F'
    return env

In [6]:
def move_player(env, pos_A, new_pos_A):
    mid_col = env.shape[1] // 2
    if env[pos_A, mid_col] != 'O': env[pos_A, mid_col] = '-'
    while env[new_pos_A, mid_col] == 'O':
        valid_pos = max(0, pos_A-2)
        new_pos_A = valid_pos
    env[new_pos_A, mid_col] = 'A'
    return env

In [7]:
def get_player(env):
    mid_col = (len(env[:][0])) // 2
    for i in range(len(env)):
        if env[i, mid_col] == 'A':
            return i
    return False

In [8]:
def move_vehicles(env):
    n_rows, n_cols = env.shape
    for row in range(1, n_rows-1):
        pos_vechicle = np.where(env[row,] == 'O')[0]
        if pos_vechicle > 0: 
            if env[row, pos_vechicle-1] == 'A': move_player(env, row, row-2)
            env[row, pos_vechicle-1] = 'O'
        else:
            if env[row, pos_vechicle-1] == 'A': move_player(env, row, row-2)
            env[row, -1] = 'O'
        env[row, pos_vechicle] = '-'
    return env

In [9]:
def is_valid_state(state, action, s_linha, n_states):
    if state+action < 0 or state+action >= n_states: 
        return False
    if state+action == s_linha: return True
    return False

In [10]:
def transition_matrix(states, actions):
    n_states = len(states)
    n_actions = len(actions)
    transition = np.zeros((n_states, n_actions, n_states))  
    
    for state in states:
        for action in range(n_actions):
            s_linha = state + actions[action]
            if (s_linha < 0 or 
            s_linha >= n_states or 
            state == (n_states)-1):
                s_linha = state
            transition[state, action, s_linha] = 1
    return transition

In [11]:
def is_terminal_state(env, pos):
    if pos == env.shape[0]-1:
        return True
    return False

In [12]:
# #Indo somente para a frente
# env = create_env(10,9)
# pos_a = get_player(env)
# score = 0
# mid_col = env.shape[1] // 2
# n_games = 20
# mean_score = 0

# for game in range(n_games):
#     score = 0
#     for i in range(272):
#         env = move_player(env, pos_a, pos_a+1)
#         env = move_vehicles(env)
#         pos_a = get_player(env)
#         if is_terminal_state(env, pos_a):
#             score += 1
#             env[0,mid_col] = 'A'
#             env[env.shape[0]-1,mid_col] = 'F'
#             pos_a = get_player(env)
#     mean_score += score

# mean_score /= n_games
# print("MEAN SCORE:", mean_score)

In [13]:
def value_iteration(states, actions, rewards, transition, V):
    while True:
        old_v = V
        for s in reversed(range(len(states))):
            v = V[s]
            max_v = 0
            for a in actions:
                v_this_action = 0
                for s_linha in reversed(range(len(states))):
                    v_this_action += rewards[states[s_linha]] + transition[s,a,s_linha]*old_v[s_linha]
                    if v_this_action > max_v:
                        max_v = v_this_action
            V[s] = max_v
        if all(old_v[s] == V[s] for s in range(len(states))): break
    return V

In [14]:
env = create_env(10,9)
pos_a = get_player(env)
score = 0
mid_col = env.shape[1] // 2
n_games = 20
mean_score = 0

mid_col = env.shape[1] // 2
time_limit = 30
actions = np.array([1, 0, -1])
rewards = {'O':-1, '-':0, 'A':0, 'I':0, 'F':1}
states = env[:,mid_col]
transition = transition_matrix(np.arange(len(env)), actions)
gamma = 0.9
epsilon = 0.001

V = [0 for _ in range(len(env))]
maior = 0

In [15]:
for game in range(n_games):
    score = 0
    for i in range(272):
        v_i = value_iteration(states, actions, rewards, transition, V)
        arg_max_act = 0
        possible_actions = []
        best_act = 0

        for pos_a_linha in range(len(states)):
            for action in actions:
                if transition[pos_a, action, pos_a_linha] != 0:
                    possible_actions.append(transition[pos_a, action, pos_a_linha])
        for possible_action in possible_actions:
            if v_i[int(pos_a + possible_action)] > arg_max_act:
                arg_max_act = v_i[int(pos_a + possible_action)]
                best_act = possible_action
        
        env = move_player(env, pos_a, int(pos_a + best_act))
        env = move_vehicles(env)
        pos_a = get_player(env)
        if is_terminal_state(env, pos_a):
            score += 1
            env[0,mid_col] = 'A'
            env[env.shape[0]-1,mid_col] = 'F'
            pos_a = get_player(env)
    mean_score += score
    print("Game", game, "- Score:", score)

mean_score /= n_games
print("MEAN SCORE:", mean_score)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
Game 0 - Score: 29
0
1